In [ ]:
import pandas as pd
import joblib
import os

# RUTAS
ruta_logs = r"C:\Users\alons\Desktop\Tesis_SIEM_ML\data\processed_logs\logs_normalizados.csv"

# Rutas a modelos y encoders ya entrenados
ruta_modelos = {
    "rf_real": r"C:\Users\alons\Desktop\Tesis_SIEM_ML\modelos\rf_modelo_supervisado_real.pkl",
    "rf_bal": r"C:\Users\alons\Desktop\Tesis_SIEM_ML\modelos\rf_modelo_supervisado_balanceado.pkl",
    "if": r"C:\Users\alons\Desktop\Tesis_SIEM_ML\modelos\modelo_isolation_forest.pkl"
}

ruta_encoders = {
    "rf_real": r"C:\Users\alons\Desktop\Tesis_SIEM_ML\modelos\label_encoders_supervisado_real.pkl",
    "rf_bal": r"C:\Users\alons\Desktop\Tesis_SIEM_ML\modelos\label_encoders_supervisado_balanceado.pkl",
    "if": r"C:\Users\alons\Desktop\Tesis_SIEM_ML\modelos\label_encoders_isolation_forest.pkl"
}

# FUNCIONES AUXILIARES

def aplicar_label_encoding_robusto(df, encoders):
    df_encoded = df.copy()
    for col, encoder in encoders.items():
        df_encoded[col] = df_encoded[col].fillna("").astype(str)
        clases = encoder.classes_
        mapa = {clase: i for i, clase in enumerate(clases)}
        df_encoded[col] = df_encoded[col].apply(lambda val: mapa.get(val, -1))
    return df_encoded

# CARGA DE LOGS
df = pd.read_csv(ruta_logs)

# PREDICCIÓN CON TODOS LOS MODELOS 
resultados = {}

for clave in ruta_modelos:
    print(f"=== Procesando modelo: {clave} ===")
    modelo = joblib.load(ruta_modelos[clave])
    encoders = joblib.load(ruta_encoders[clave])

    df_copia = df.copy()
    df_encoded = aplicar_label_encoding_robusto(df_copia, encoders)
    df_encoded = df_encoded[list(encoders.keys())]

    # Hacer predicción
    df[f"pred_{clave}"] = modelo.predict(df_encoded)
    resultados[clave] = df[f"pred_{clave}"].value_counts()

print("Resumen de predicciones:")
for clave, valores in resultados.items():
    print(f"\nModelo: {clave}")
    print(valores)

# (Opcional) Guardar el resultado con las predicciones
df.to_csv(r"C:\Users\alons\Desktop\Tesis_SIEM_ML\results\resultados_predicciones.csv", index=False, encoding="utf-8-sig")
print("\nArchivo guardado como 'resultados_predicciones.csv'")

C:\Users\alons\AppData\Local\Temp\ipykernel_18404\1426998904.py:33: DtypeWarning: Columns (6,7,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta_logs)


=== Procesando modelo: rf_real ===
=== Procesando modelo: rf_bal ===
=== Procesando modelo: if ===
Resumen de predicciones:

Modelo: rf_real
pred_rf_real
1    675632
0    181176
Name: count, dtype: int64

Modelo: rf_bal
pred_rf_bal
1    675632
0    181176
Name: count, dtype: int64

Modelo: if
pred_if
 1    836477
-1     20331
Name: count, dtype: int64

Archivo guardado como 'resultados_predicciones.csv'


In [ ]:
# Mostrar ejemplos concretos de predicción
print("\n=== EJEMPLOS DE PREDICCIONES ===")
columnas_pred = ['pred_rf_real', 'pred_rf_bal', 'pred_if']
columnas_mostrar = columnas_pred + [col for col in df.columns if col not in columnas_pred]
df_ejemplos = df[columnas_mostrar].head(5)
print(df_ejemplos.to_string(index=False))

# Mostrar ejemplos positivos detectados por RF real
print("\n=== EJEMPLOS POSITIVOS DETECTADOS POR RF REAL ===")
ej_rf_real = df[df['pred_rf_real'] == 1][columnas_mostrar].head(5)
print(ej_rf_real.to_string(index=False))

# Mostrar ejemplos positivos detectados por RF balanceado
print("\n=== EJEMPLOS POSITIVOS DETECTADOS POR RF BALANCEADO ===")
ej_rf_bal = df[df['pred_rf_bal'] == 1][columnas_mostrar].head(5)
print(ej_rf_bal.to_string(index=False))

# Mostrar ejemplos positivos detectados por Isolation Forest
print("\n=== EJEMPLOS POSITIVOS DETECTADOS POR ISOLATION FOREST ===")
ej_if = df[df['pred_if'] == 1][columnas_mostrar].head(5)
print(ej_if.to_string(index=False))

# Mostrar ejemplos donde los modelos no coinciden
print("\n=== EJEMPLOS DONDE LOS MODELOS NO COINCIDEN ===")
df_diferencias = df[
    (df['pred_rf_real'] != df['pred_rf_bal']) |
    (df['pred_rf_real'] != df['pred_if']) |
    (df['pred_rf_bal'] != df['pred_if'])
]
df_diferencias_mostrar = df_diferencias[columnas_mostrar].head(5)
print(df_diferencias_mostrar.to_string(index=False))

# Guardar ejemplos discrepantes en un archivo CSV para revisión adicional
# df_diferencias_mostrar.to_csv("C:/Users/alons/Desktop/Tesis_SIEM_ML/data/ejemplos_discrepantes.csv", index=False)


=== EJEMPLOS DE PREDICCIONES ===
 pred_rf_real  pred_rf_bal  pred_if                   timestamp         src_user                     action                             description   object_type                                   object_name  channel log_level      log_source src_ip dst_ip protocol event_id severity
            0            0        1 May 10, 2025 @ 15:47:04.800 60127-099940047$ object-operation-performed   Se realizó una operación en un objeto WMI Namespace root\cimv2\Security\MicrosoftVolumeEncryption Security      info 60127-099940047    NaN    NaN      NaN      NaN      NaN
            0            0        1 May 10, 2025 @ 15:47:04.800 60127-099940047$ object-operation-performed   Se realizó una operación en un objeto WMI Namespace root\cimv2\Security\MicrosoftVolumeEncryption Security      info 60127-099940047    NaN    NaN      NaN      NaN      NaN
            0            0        1 May 10, 2025 @ 15:47:04.798 60127-099940047$ object-operation-performed   Se r

In [12]:
print("\n=== INFORMES DETALLADOS CON EXPLICACIÓN DE ANOMALÍA ===")

# Columnas necesarias
columnas_pred = ['pred_rf_real', 'pred_rf_bal', 'pred_if']
columnas_mostrar = columnas_pred + [
    col for col in df.columns if col in ['timestamp', 'src_user', 'action', 'description', 'object_type', 'channel', 'log_level']
]

# Selección de ejemplos
df_ejemplos = df[columnas_mostrar].head(5)

# Revisión de cada evento
for idx, row in df_ejemplos.iterrows():
    print(f"\n EVENTO {idx+1} - ANÁLISIS DETALLADO")
    print(f" Fecha y hora: {row['timestamp']}")
    print(f" Usuario: {row['src_user']}")
    print(f" Acción: {row['action']}")
    print(f" Descripción: {row['description']}")
    print(f" Objeto afectado: {row['object_type']}")
    print(f" Canal: {row['channel']}")
    print(f" Nivel del log: {row['log_level']}")

    print("\n RESPUESTA DE LOS MODELOS:")

    # Random Forest Real
    if row['pred_rf_real'] == 1:
        print(" [RF Real] Detectó posible AMENAZA basada en datos reales.")
    else:
        print(" [RF Real] No considera este evento riesgoso.")

    # Random Forest Balanceado
    if row['pred_rf_bal'] == 1:
        print(" [RF Balanceado] También lo detectó como AMENAZA.")
    else:
        print(" [RF Balanceado] Evento clasificado como normal.")

    # Isolation Forest
    if row['pred_if'] in [1, -1]:
        print(" [Isolation Forest] Detectó una ANOMALÍA.")
        print(" Posible explicación:")

        # Explicación basada en heurística
        if pd.isna(row['src_user']) or '$' in str(row['src_user']):
            print("    • El evento fue ejecutado por una cuenta de sistema o servicio, lo cual puede ocultar actividad maliciosa.")
        if "credential" in str(row['action']).lower():
            print("    • Se accedió al gestor de credenciales, lo que suele ser inusual en logs normales.")
        if "permissions" in str(row['action']).lower():
            print("    • Hubo un cambio de permisos, lo cual puede estar asociado a escalamiento de privilegios.")
        if row['log_level'] in ['critical', 'error']:
            print("    • El nivel del log es alto, indicando un posible incidente o fallo importante.")
        if row['channel'] not in ['System', 'Security']:
            print("    • El evento fue registrado en un canal poco común.")
        if str(row['object_type']).strip() == '-' or pd.isna(row['object_type']):
            print("    • No se especifica el tipo de objeto afectado, lo cual puede indicar evasión de registro.")

    else:
        print(" [Isolation Forest] No detectó ninguna anomalía.")

    print("\n CONCLUSIÓN:")
    if row['pred_rf_real'] == 1 or row['pred_rf_bal'] == 1 or row['pred_if'] in [1, -1]:
        print(" Este evento requiere revisión humana. Múltiples señales apuntan a comportamiento sospechoso.")
    else:
        print(" El evento es considerado normal por todos los modelos.")

print("\n Fin del informe detallado.")


=== INFORMES DETALLADOS CON EXPLICACIÓN DE ANOMALÍA ===

 EVENTO 1 - ANÁLISIS DETALLADO
 Fecha y hora: May 10, 2025 @ 15:47:04.800
 Usuario: 60127-099940047$
 Acción: object-operation-performed
 Descripción: Se realizó una operación en un objeto
 Objeto afectado: WMI Namespace
 Canal: Security
 Nivel del log: info

 RESPUESTA DE LOS MODELOS:
 [RF Real] No considera este evento riesgoso.
 [RF Balanceado] Evento clasificado como normal.
 [Isolation Forest] Detectó una ANOMALÍA.
 Posible explicación:
    • El evento fue ejecutado por una cuenta de sistema o servicio, lo cual puede ocultar actividad maliciosa.

 CONCLUSIÓN:
 Este evento requiere revisión humana. Múltiples señales apuntan a comportamiento sospechoso.

 EVENTO 2 - ANÁLISIS DETALLADO
 Fecha y hora: May 10, 2025 @ 15:47:04.800
 Usuario: 60127-099940047$
 Acción: object-operation-performed
 Descripción: Se realizó una operación en un objeto
 Objeto afectado: WMI Namespace
 Canal: Security
 Nivel del log: info

 RESPUESTA DE LO